In [2]:
import pandas as pd
import numpy as np
import requests as rqst
from bs4 import BeautifulSoup as bs

import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import plotly.graph_objects as go
import time




# Set up chrome webdriver. chromedriver.exe must be in the same folder as the file
driver = uc.Chrome()

# Input username and password. this could be pulled from a separate file for anonymity in the future. Currently 
username = 'your username'
password = 'your password'
google_username = 'your username'
google_password = 'your password'

# WAS FORCED TO SIGN INTO GOOGLE TO AVOID 2 FACTOR AUTHENTIFICATION ON ESPN. 

# Navigate to google
driver.get('https://google.com')

# Sign in

# Elect Sign in tab
WebDriverWait(driver, 25).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "Sign in"))
    )

google_link = driver.find_element(By.LINK_TEXT, "Sign in")
google_link.click()

# enter email
WebDriverWait(driver, 25).until(
    EC.presence_of_element_located((By.ID, "identifierId"))
)

google_sign_in = driver.find_element(By.ID, "identifierId").send_keys(google_username + Keys.ENTER)

# enter password
WebDriverWait(driver, 25).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input"))
)

google_password = driver.find_element(By.CSS_SELECTOR, "#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input").send_keys(google_password +Keys.ENTER)

# Navigate to the scoreboard page
driver.get('https://fantasy.espn.com/baseball/league/scoreboard?leagueId=16292')

# Wait for the iframe to be available and switch to it
WebDriverWait(driver, 25).until(
    EC.presence_of_element_located((By.TAG_NAME, "iframe")) 
)
iframe = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe)

# Wait for the username field to be available within the iframe
WebDriverWait(driver, 25).until(
    EC.presence_of_element_located((By.ID, "InputIdentityFlowValue"))
)

# Select the username input element and enter the username, then submit by hitting ENTER
username_input = driver.find_element(By.ID, "InputIdentityFlowValue").send_keys(username + Keys.ENTER)


# Wait for the password field to become available within the iframe
WebDriverWait(driver, 25).until(
    EC.presence_of_element_located((By.ID, "InputPassword"))
)

# Now enter the password and submit
password_input = driver.find_element(By.ID, "InputPassword").send_keys(password + Keys.ENTER)


# Switch out of the iframe and back into the main document
driver.switch_to.default_content()

# Wait for the dropdown to be available and select it
WebDriverWait(driver, 25).until(
    EC.presence_of_element_located((By.CLASS_NAME, "dropdown__select"))
)
dropdown = driver.find_element(By.CLASS_NAME, "dropdown__select")

# Create a Select object
select = Select(dropdown)

dropdown.click()

# Get all options in the dropdown
options = select.options

# Find the current index for the weekly matchup
current_index = None
for index, option in enumerate(options):
    if option.is_selected():
        current_index = index
        break
        
# Instantiate Dataframe that will hold all of the data 
cumulative_opposing_team_data = pd.DataFrame()       
        
# loop through indexes and scrape data
for i in range(current_index):


    # Select the dropdown at index i  and wait for data to load  
    select.select_by_index(i)
    time.sleep(2)
    
    # Locate the html and pass it into beautiful soup for scraping
    html = driver.page_source

    soup = bs(html, 'html.parser')

    #Extract the data from each table
    data_frames = []
    scoreboard_records = []
    
    tables = soup.find_all('table', class_='Table')  
    scores = soup.find_all('div', class_= 'ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2')

    # Pull the results from the score table 
    for table in tables:
        # Extracting all rows in the table
        rows = table.find_all('tr')
        table_data = []
        for row in rows:
            cols = row.find_all('td')  # or 'th' if you want headers
            cols = [element.text.strip() for element in cols]
            table_data.append(cols)
        # Create a DataFrame for the current table
        df = pd.DataFrame(table_data)
        data_frames.append(df)


    # Clean Data_frames and swap team names per match so that the data is opponent
    Headers = ['Team_Name','R','HR','TB','RBI','K','SB','AVG','OPS','IP','BB','K_pitch','QS','ERA','WHIP','SVHD']

    for i in range(len(data_frames)):
        # Get the last two rows of each DataFrame
        data_frames[i] = data_frames[i].tail(2)

        # Assign new headers
        data_frames[i].columns = Headers

        # Swap the 'Team_Name' values between the first and second rows
        name0 = data_frames[i].iat[0, 0]
        name1 = data_frames[i].iat[1, 0]
        data_frames[i].iat[0, 0] = name1
        data_frames[i].iat[1, 0] = name0
    

    # combine cleaned dfs into one df for the week
    weekly_df = pd.concat(data_frames, ignore_index=True)
    
    # pull the final results from each matchup and add it to the weekly df
    
    # write a function that changes the order of the scores. so that the scores match the teams
    def swap_scores(scores):
        swapped_scores = scores.copy()
        for i in range(0,len(scores)-1,2):
            swapped_scores[i], swapped_scores[i+1] = swapped_scores[i+1], swapped_scores[i]
        return swapped_scores

    # use function to create a swapped list of scores
    score_reordered = []
    for score in swap_scores(scores):
        score_reordered.append(score.text)
        
    # Add the list as a new column in the dataframe
    weekly_df['Cumalitive_record'] = score_reordered
        

    # combine weekly data with previous weeks
    cumulative_opposing_team_data = pd.concat([cumulative_opposing_team_data, weekly_df], ignore_index=True)
    
# Add wins, Loss and Tie Columns
cumulative_opposing_team_data[['Wins','Losses','Draws']] = cumulative_opposing_team_data['Cumalitive_record'].str.split('-', expand = True).astype(int)

# drop the cumalative record columns
cumulative_opposing_team_data  = cumulative_opposing_team_data.drop(columns = ['Cumalitive_record'])

# Change column types
int_cols = ['R','HR','TB','RBI','K','SB','BB','K_pitch','QS','SVHD']
float_cols = ['AVG','OPS','IP','ERA','WHIP']

for col in int_cols:
    cumulative_opposing_team_data[col] = cumulative_opposing_team_data[col].astype(int)
    
for col in float_cols:
     cumulative_opposing_team_data[col] = cumulative_opposing_team_data[col].astype(float)
        
# Make IP col converted to deciamls
def convert_ip(ip):
    full_inning = int(ip)
    part_ining = round(ip - full_inning, 1)
    decimal = 0
    
    if part_ining == 0.1:
        decimal = 1/3
    elif part_ining == 0.2:
        decimal = 2/3
        
    return float(full_inning) + decimal


cumulative_opposing_team_data['IP_conv'] = round(cumulative_opposing_team_data['IP'].apply(convert_ip),1)
cumulative_opposing_team_data['R_allowed'] = round(cumulative_opposing_team_data['ERA']*cumulative_opposing_team_data['IP_conv']/9, 1)
cumulative_opposing_team_data['W_H_allowed'] = round(cumulative_opposing_team_data['WHIP']*cumulative_opposing_team_data['IP_conv'], 1)

# Aggregate the data
aggregated_df = cumulative_opposing_team_data.groupby('Team_Name').agg({
    'R': 'sum',        
    'HR': 'sum',       
    'TB': 'sum',       
    'RBI': 'sum',      
    'K': 'sum',       
    'SB': 'sum',       
    'AVG': lambda x: round(x.mean(), 4),    
    'OPS': lambda x: round(x.mean(), 4),      
    'IP': 'sum',       
    'BB': 'sum',
    'K_pitch': 'sum',
    'ERA': 'mean',     
    'WHIP': 'mean',    
    'SVHD': 'sum',
    'R_allowed': 'sum',
    'W_H_allowed': 'sum',
    'IP_conv': 'sum',
    'Wins': 'sum',
    'Losses': 'sum',
    'Draws': 'sum'
})

# function to change IPs back to original format

def conv_ip_back(ip):
    full_inning = int(ip)
    partial_inning = ip - full_inning
    
    if partial_inning == 0:
        partial_inning = 0
    elif 0.4 >= partial_inning > 0:
        partial_inning = .1
    elif 0.8 >= partial_inning > 0.4:
        partial_inning = 0.2
    elif partial_inning > 0.8:
        partial_inning = 1
    return float(full_inning) + partial_inning
    
# Recalculate WHIP AND ERA and IP drop the old Columns
aggregated_df['IP'] = round(aggregated_df['IP_conv'].apply(conv_ip_back),1)
aggregated_df['ERA'] = round((aggregated_df['R_allowed']*9/aggregated_df['IP_conv']),3)
aggregated_df['WHIP'] = round(aggregated_df['W_H_allowed']/aggregated_df['IP_conv'], 3)
aggregated_df = aggregated_df.drop(columns=['R_allowed', 'W_H_allowed', 'IP_conv'])

# run Dash to Output a sortable table

# Import necessary components from Dash library
import dash
from dash import html, dcc, dash_table  # Corrected import for Dash 1.0+
import pandas as pd

# Reset the index so that the team names a coolumn
aggregated_df.reset_index(inplace=True)

# Initialize the Dash app
app = dash.Dash(__name__)  # Make sure this is dash.Dash

# Define the layout of the app using Dash HTML components
app.layout = html.Div([
    html.H1('Stats Against', style={'textAlign':'center'}),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in aggregated_df.columns],  
        data=aggregated_df.to_dict('records'),  
        sort_action="native",  
#         style_table={'height': '300px', 'overflowY': 'auto'}
    )
], style = {'padding' : '20px'})

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.78)
Stacktrace:
	GetHandleVerifier [0x00D1C113+48259]
	(No symbol) [0x00CACA41]
	(No symbol) [0x00BA0A17]
	(No symbol) [0x00B7E02B]
	(No symbol) [0x00C0742E]
	(No symbol) [0x00C19476]
	(No symbol) [0x00C00B36]
	(No symbol) [0x00BD570D]
	(No symbol) [0x00BD62CD]
	GetHandleVerifier [0x00FD6533+2908323]
	GetHandleVerifier [0x01013B4B+3159739]
	GetHandleVerifier [0x00DB505B+674763]
	GetHandleVerifier [0x00DBB21C+699788]
	(No symbol) [0x00CB6244]
	(No symbol) [0x00CB2298]
	(No symbol) [0x00CB242C]
	(No symbol) [0x00CA4BB0]
	BaseThreadInitThunk [0x75A67BA9+25]
	RtlInitializeExceptionChain [0x774BBE3B+107]
	RtlClearBits [0x774BBDBF+191]
